# LGMC (dynamic) - Solid Electrolyte Interphase

Simple example of solid-electrolyte interphase formation and evolution in a lithium-ion battery.

In [2]:
import sqlite3
from itertools import product, combinations
import copy
import os
import pandas as pd

import numpy as np
from scipy.constants import pi, epsilon_0, elementary_charge

from monty.serialization import loadfn

In [3]:
# Room temperature (25 C) in Kelvin
ROOM_TEMP = 298.15

# Boltzmann constant in eV / K
KB = 8.617333262 * 10 ** -5

# Planck constant in eV * s
PLANCK = 4.135667696 * 10 ** -15

# Simulation variables
dielectric = 18.5
refractive = 1.415
radius = 5.0
electrode_distance = 0.0
adiabatic=True
decay_constant=1.2

# kMC parameters
factor_zero = 1.0
factor_two = 1.0
factor_duplicate = 0.5

In [4]:
# Functions
def eyring(dg_barrier, kappa=1.0, temperature=ROOM_TEMP):
    if dg_barrier <= 0:
        return kappa * KB * temperature / PLANCK
    else:
        return kappa * KB * temperature / PLANCK * np.exp(-dg_barrier / (KB * temperature))


def lambda_outer(r=radius, d=electrode_distance, eps=dielectric, n=refractive):
    lambda_o = abs(elementary_charge) / (8 * pi * epsilon_0)
    lambda_o *= (1 / r - 1 / (2 * d)) * 10 ** 10
    lambda_o *= 1 / n ** 2 - 1 / eps
    return lambda_o


def barrier_redox(dg, lambda_i, dq=1, e_free=-1.4, r=radius, d=electrode_distance, eps=dielectric, n=refractive):
    lambda_o = lambda_outer(r=r, d=d, eps=eps, n=n)
    lambda_total = lambda_i + lambda_o

    delta_g = dg + dq * e_free

    dg_barrier = lambda_total / 4 * (1 + delta_g / lambda_total) ** 2

    return dg_barrier

In [5]:
def make_df(path, table):
    # Read sqlite query results into a pandas DataFrame
    con = sqlite3.connect(path)
    df = pd.read_sql_query("SELECT * FROM " + table, con)
    con.close()
    
    return df 

In [37]:
def create_state_sqlite(path, stmts, num_species, initial_state,
                        insert_into_factors = "INSERT INTO factors VALUES (?,?,?)",
                        factor_zero = 1, factor_two = 1, factor_duplicate = 0.5):
    '''Create sqlite of initial species concentrations'''
    
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()
    
    for stmt in stmts:    
        rn_cur.execute(stmt)
    rn_con.commit()

    rn_cur.execute(
        insert_into_factors,
        (factor_zero, factor_two, factor_duplicate)
    )

    for i in range(num_species):
        rn_cur.execute(
            "INSERT INTO initial_state VALUES (?,?)",
            (i, initial_state.get(i,0)))

    rn_con.commit()
    rn_con.close()

In [7]:
def create_reaction_sqlite(path, stmts, num_species, 
                           insert_metadata = """ INSERT INTO metadata VALUES (?, ?)"""):
    ''' Create an empty sqlite, name, with a table for the metadata and 
    reactions'''
    
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()
    
    for stmt in stmts:    
        rn_cur.execute(stmt)
    rn_con.commit()
    
    rn_cur.execute(
        insert_metadata,
        (num_species, 0))

    rn_con.commit()
    
    rn_con.close()

In [8]:
def add_homogeneous_reactions(path, chem_rxns, species_thermo, mapping, insert_reaction, phase):
    ''' Add homogeneous chemical lattice reactions to specified table'''
    
    # connect to table
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()

    df = make_df(path, "metadata")
    reaction_index = int(df['number_of_reactions'][0])

    reactions = set()
    print("ADDING CHEMICAL REACTIONS")

    for rxn in chem_rxns:
        rcts = rxn["reactants"]
        pros = rxn["products"]

        g_rct = sum([species_thermo[x]['g'] for x in rcts])
        g_pro = sum([species_thermo[x]['g'] for x in pros])
        dg_forward = round(g_pro - g_rct, 2)
        dg_reverse = -1 * dg_forward

        num_reactants = len(rcts)
        num_products = len(pros)

        indices_reactants = [mapping[x] for x in rcts]
        if num_reactants == 1:
            indices_reactants.append(-1)

        indices_products =  [mapping[x] for x in pros]
        if num_products == 1:
            indices_products.append(-1)

        barrier_forward = rxn["barrier"]
        barrier_reverse = barrier_forward - dg_forward

        k_forward = eyring(barrier_forward)
        k_reverse = eyring(barrier_reverse)

        if (indices_reactants[0], indices_reactants[1], indices_products[0], indices_products[1]) not in reactions:
            reactions.add((indices_reactants[0], indices_reactants[1], indices_products[0], indices_products[1]))

            reactant_two = indices_reactants[1]
            product_two = indices_products[1]
            num_react = num_reactants
            num_prod = num_products
            
            if(phase == 'L'):           
                if(num_reactants > num_products):
                    reactant_two = indices_reactants[1]
                    product_two = 0
                    num_prod = 2
                elif(num_reactants < num_products):
                    reactant_two = 0
                    product_two = indices_products[1]
                    num_react = 2
            
            rn_cur.execute(insert_reaction,
                       (reaction_index,
                        num_react,
                        num_prod,
                        indices_reactants[0],
                        reactant_two,
                        indices_products[0],
                        product_two,
                        phase,
                        phase,
                        phase,
                        phase,
                        dg_forward,
                        1,
                        k_forward,
                        0,
                        0,
                        0,
                        phase))
                

            reaction_index += 1
            rn_con.commit()

        if (indices_products[0], indices_products[1], indices_reactants[0], indices_reactants[1]) not in reactions:
            reactions.add((indices_products[0], indices_products[1], indices_reactants[0], indices_reactants[1]))

            
            reactant_two = indices_reactants[1]
            product_two = indices_products[1]
            num_react = num_reactants
            num_prod = num_products
            
            if(phase == 'L'):
                if(num_reactants > num_products):
                    reactant_two = indices_reactants[1]
                    product_two = 0
                    num_prod = 2
                elif(num_reactants < num_products):
                    reactant_two = 0
                    product_two = indices_products[1]
                    num_react = 2
            
            
            rn_cur.execute(insert_reaction,
                           (reaction_index,
                            num_prod,
                            num_react,
                            indices_products[0],
                            product_two,
                            indices_reactants[0],
                            reactant_two,
                            phase,
                            phase,
                            phase,
                            phase,
                            dg_reverse,
                            1,
                            k_reverse,
                            0,
                            0,
                            0,
                            phase))
            reaction_index += 1
            rn_con.commit()

    print("DONE ADDING CHEMICAL REACTIONS; {} TOTAL REACTIONS".format(reaction_index))

    rn_cur.execute("""Update metadata set number_of_reactions = """ + str(reaction_index))

    rn_con.commit()
    rn_con.close()
    

In [9]:
def add_adsorption(path, adsorp_rxns, mapping, insert_reaction):
    '''Add adsorption reaction to sqlite file. adsorp_rnxs 2D array
    of species and corresponding rate [species, rate]'''
    
    # connect to table
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()

    df = make_df(path, "metadata")
    reaction_index = int(df['number_of_reactions'][0])
    
    for rnx in adsorp_rxns:
        
        rn_cur.execute(insert_reaction,
                           (reaction_index,
                            2,
                            1,
                            0,
                            mapping[rnx[0]],
                            mapping[rnx[0]],
                            -1,
                            "L",
                            "S",
                            "L",
                            "N",
                            -100,
                            1,
                            rnx[1],
                            0,
                            0,
                            0,
                            "A"))
        reaction_index += 1
        rn_con.commit()

    print("DONE ADDING ADSORPTION REACTIONS; {} TOTAL REACTIONS".format(reaction_index))
    
    rn_cur.execute("""Update metadata set number_of_reactions = """ + str(reaction_index))

    rn_con.commit()
    rn_con.close()    

In [10]:
def add_desorption(path, desorp_rxns, mapping, insert_reaction):
    '''Add adsorption reaction to sqlite file. adsorp_rnxs 2D array
    of species and corresponding rate [species, rate]'''
    
    # connect to table
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()

    df = make_df(path, "metadata")
    reaction_index = int(df['number_of_reactions'][0])
    
    for rnx in desorp_rxns:
        
        rn_cur.execute(insert_reaction,
                           (reaction_index,
                            1,
                            2,
                            mapping[rnx[0]],
                            -1,
                            0,
                            mapping[rnx[0]],
                            "L",
                            "N",
                            "L",
                            "S",
                            -100,
                            1,
                            rnx[1],
                            0,
                            0,
                            0,
                            "D"))
        reaction_index += 1
        rn_con.commit()

    print("DONE ADDING DESORPTION REACTIONS; {} TOTAL REACTIONS".format(reaction_index))
    
    rn_cur.execute("""Update metadata set number_of_reactions = """ + str(reaction_index))

    rn_con.commit()
    rn_con.close() 

In [11]:
def add_diffusion(path, diff_rxns, mapping, insert_reaction):
    ''' Add diffusion reactions to sqlite file (name). diff_rxns 2D array of 
        species, rate'''
    
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()

    df = make_df(path, "metadata")
    reaction_index = int(df['number_of_reactions'][0])

    for rnx in diff_rxns:

        rn_cur.execute(insert_reaction,
                       (reaction_index,
                        2,
                        2,
                        mapping[rnx[0]],
                        0,
                        0,
                        mapping[rnx[0]],
                        "L",
                        "L",
                        "L",
                        "L",
                        0,
                        1,
                        rnx[1],
                        0,
                        0,
                        0,
                        "F"))
        reaction_index += 1
        rn_con.commit()

    print("DONE ADDING DIFFUSION REACTIONS; {} TOTAL REACTIONS".format(reaction_index))

    rn_cur.execute("""Update metadata set number_of_reactions = """ + str(reaction_index))

    rn_con.commit()
    rn_con.close() 
    

In [12]:
def add_reduction(path, rxns, mapping, insert_reaction, prefactor):
    ''' Add reduction reactions to sqlite file (name). rxns '''
    
    rn_con = sqlite3.connect(path)
    rn_cur = rn_con.cursor()

    df = make_df(path, "metadata")
    reaction_index = int(df['number_of_reactions'][0])

    for rnx in rxns:

        rn_cur.execute(insert_reaction,
                       (reaction_index,
                        1,
                        1,
                        mapping[rnx['reactants'][0]],
                        -1,
                        mapping[rnx['products'][0]],
                        -1,
                        "L",
                        "N",
                        "L",
                        "N",
                        rnx['dg'],
                        prefactor,
                        0,
                        1.2,
                        0.32 + rnx['lambda_inner'],
                        0.5,
                        "R"))
        reaction_index += 1
        rn_con.commit()
        
        rn_cur.execute(insert_reaction,
                       (reaction_index,
                        1,
                        1,
                        mapping[rnx['reactants'][0]],
                        -1,
                        mapping[rnx['products'][0]],
                        -1,
                        "S",
                        "N",
                        "S",
                        "N",
                        rnx['dg'],
                        prefactor,
                        0,
                        1.2,
                        0.32 + rnx['lambda_inner'],
                        0.5,
                        "R"))
        reaction_index += 1
        rn_con.commit()

    print("DONE ADDING REDUCTION REACTIONS; {} TOTAL REACTIONS".format(reaction_index))

    rn_cur.execute("""Update metadata set number_of_reactions = """ + str(reaction_index))

    rn_con.commit()
    rn_con.close() 
    

In [13]:
create_metadata_table = """
    CREATE TABLE metadata (
            number_of_species   INTEGER NOT NULL,
            number_of_reactions INTEGER NOT NULL
    );
"""

insert_metadata = """
    INSERT INTO metadata VALUES (?, ?)
"""

# it is important that reaction_id is the primary key
# otherwise the network loader will be extremely slow.
create_reactions_table = """
    CREATE TABLE reactions (
            reaction_id                     INTEGER NOT NULL PRIMARY KEY,
            number_of_reactants             INTEGER NOT NULL,
            number_of_products              INTEGER NOT NULL,
            reactant_1                      INTEGER NOT NULL,
            reactant_2                      INTEGER NOT NULL,
            product_1                       INTEGER NOT NULL,
            product_2                       INTEGER NOT NULL,
            phase_reactant_1                CHAR(1) NOT NULL,
            phase_reactant_2                CHAR(1) NOT NULL,
            phase_product_1                 CHAR(1) NOT NULL,
            phase_product_2                 CHAR(1) NOT NULL,
            dG                              REAL NOT NULL,
            prefactor                       REAL NOT NULL,
            rate                            REAL NOT NULL,
            electron_tunneling_coefficient  REAL NOT NULL,
            reorganization_energy           REAL NOT NULL,
            charge_transfer_coefficient     REAL NOT NULL,
            type                            CHAR(1) NOT NULL
    );
"""

insert_reaction = """
    INSERT INTO reactions VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

create_initial_state_table = """
    CREATE TABLE initial_state (
            species_id             INTEGER NOT NULL PRIMARY KEY,
            count                  INTEGER NOT NULL
    );
"""
create_trajectories_table = """
    CREATE TABLE trajectories (
            seed                INTEGER NOT NULL,
            step                INTEGER NOT NULL,
            time                REAL NOT NULL,
            reaction_id         INTEGER NOT NULL,
            site_1_mapping      INTEGER NOT NULL,
            site_2_mapping      INTEGER NOT NULL
    );
"""

create_factors_table = """
    CREATE TABLE factors (
            factor_zero      REAL NOT NULL,
            factor_two       REAL NOT NULL,
            factor_duplicate REAL NOT NULL
    );
"""

create_interrupt_state = """
    CREATE TABLE interrupt_state (
            seed                    INTEGER NOT NULL,
            species_id              INTEGER NOT NULL,
            quantity                INTEGER NOT NULL,
            site_mapping            INTEGER NOT NULL,
            edge                    INTEGER NOT NULL
            
    );
"""

create_interrupt_cutoff = """
    CREATE TABLE interrupt_cutoff (
            seed                    INTEGER NOT NULL,
            step                    INTEGER NOT NULL,
            time                    INTEGER NOT NULL, 
            maxk                   INTEGER NOT NULL
            
    );
"""

In [14]:
species_thermo = loadfn("test_species_thermo.json")
chem_rxns = loadfn("test_energy_barriers.json")
echem_rxns = loadfn("test_lambda_inner.json")
hopping_reactions = loadfn("test_hopping_reactions.json")

mapping = {m: i for i, m in enumerate(species_thermo, 1)}
chem_rxns[2]['products'] = ['C2H40', 'LiCO3-']

In [39]:
# Create sqlite file with initial state
initial_state = {mapping["LiEC+"]: 10000}

stmts = [create_initial_state_table, create_trajectories_table, 
         create_factors_table, create_interrupt_cutoff, 
         create_interrupt_state]

# Create initial state database 
create_state_sqlite("initial_state.sqlite", stmts, len(species_thermo)+1, initial_state)

In [22]:
# Create reaction database and add reactions
stmts = [create_metadata_table, create_reactions_table]
create_reaction_sqlite('network.sqlite', stmts, len(species_thermo)+1)

In [28]:
# Add homogeneous reactions to reaction.sqlite
# All reactions can occur in either phase
add_homogeneous_reactions('network.sqlite', chem_rxns, species_thermo, mapping, insert_reaction, 'S')
add_homogeneous_reactions('network.sqlite', chem_rxns, species_thermo, mapping, insert_reaction, 'L')
add_homogeneous_reactions('network.sqlite', hopping_reactions, species_thermo, mapping, insert_reaction, 'L')
add_homogeneous_reactions('network.sqlite', hopping_reactions, species_thermo, mapping, insert_reaction, 'S')

ADDING CHEMICAL REACTIONS
DONE ADDING CHEMICAL REACTIONS; 12 TOTAL REACTIONS
ADDING CHEMICAL REACTIONS
DONE ADDING CHEMICAL REACTIONS; 24 TOTAL REACTIONS
ADDING CHEMICAL REACTIONS
DONE ADDING CHEMICAL REACTIONS; 54 TOTAL REACTIONS
ADDING CHEMICAL REACTIONS
DONE ADDING CHEMICAL REACTIONS; 84 TOTAL REACTIONS


In [31]:
# Add adsorption reactions
# Species which can adsorb: LiCO3-, Li2CO30, LEDC0, LEDC-, LEDC_minus_Li-, LEDC_plus_Li+, LEDC_plus_Li0
x = 1e4
rxns = [["LiCO3-", x], ["Li2CO30", x], ["LEDC0", x], ["LEDC-", x], ["LEDC_minus_Li-", x],
           ["LEDC_plus_Li+", x], ["LEDC_plus_Li0", x]]
add_adsorption('network.sqlite', rxns, mapping, insert_reaction)

DONE ADDING ADSORPTION REACTIONS; 91 TOTAL REACTIONS


In [32]:
rxns = [["LiCO3-", 1e1], ["C2H40", x]]
add_desorption('network.sqlite', rxns, mapping, insert_reaction)

DONE ADDING DESORPTION REACTIONS; 93 TOTAL REACTIONS


In [33]:
prefactor = 1e6
add_reduction('network.sqlite', echem_rxns, mapping, insert_reaction, prefactor)

DONE ADDING REDUCTION REACTIONS; 99 TOTAL REACTIONS
